## Importing libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests

## Using selenium for automation

In [3]:
driver = webdriver.Chrome(executable_path= 'C:\Windows\chromedriver.exe')
driver.get('https://imdb.com')

# maximize window
driver.maximize_window()

# dropdown
dropdown = driver.find_element_by_class_name('ipc-icon--arrow-drop-down')
dropdown.click()
time.sleep(1)

# click in advanced search from dropdown menu
element = driver.find_element_by_link_text('Advanced Search')
element.click()

# click on advanced title search
adv_sch = driver.find_element_by_link_text('Advanced Title Search')
adv_sch.click()

# select feature films
feature_film = driver.find_element_by_id("title_type-1")
feature_film.click()

# select tv movie
tv_movie = driver.find_element_by_id("title_type-2")
tv_movie.click()

# release date min
date_min = driver.find_element_by_name("release_date-min")
date_min.click()
date_min.send_keys('1990')

# release date max
date_max = driver.find_element_by_name("release_date-max")
date_max.click()
date_max.send_keys('2021')

# rating min
rate_min = driver.find_element_by_name("user_rating-min")
rate_min.click()
dropdown2 = Select(rate_min)
dropdown2.select_by_visible_text('1.0')

# rating max
rate_max = driver.find_element_by_name("user_rating-max")
rate_max.click()
dropdown3 = Select(rate_max)
dropdown3.select_by_visible_text('10')

# oscar nominated
oscar_nom = driver.find_element_by_id("groups-7")
oscar_nom.click()

# color 
color = driver.find_element_by_id("colors-1")
color.click()

# language
language = driver.find_element_by_name("languages")
dropdown4 = Select(language)
dropdown4.select_by_visible_text('English')

# select 250 per page
count_per_page = driver.find_element_by_id("search-count")
count_per_page.click()
dropdown5 = Select(count_per_page)
dropdown5.select_by_index(2)

# click on search button
search_button = driver.find_element_by_xpath('(//button[@type="submit"])[2]')
search_button.click()

# url
url = driver.current_url

## Data extraction with BeautifulSoup

In [5]:
# get request
response = requests.get(url)

In [6]:
# status code
response.status_code

200

In [7]:
# soup object
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film/TV Movie,
Released between 1990-01-01 and 2021-12-31,
User Rating between 1 and 10,
Oscar-Nominated,
Color,
English
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
<

In [9]:
# Starting point for result items
list_items = soup.find_all("div", {"class": "lister-item"})

In [10]:
len(list_items)

250

### Attributes of the data to be extracted

- movie
- year
- duration
- genre
- rating

In [14]:
# Extracting the movie title
list_items[0].find('h3').find('a').get_text()

'The Nightmare Before Christmas'

In [19]:
# Extracting the year
list_items[0].find('h3').find('span', {"class": "lister-item-year"}).get_text().replace('(', '').replace(')', '')  

'1993'

In [21]:
# Extracting the duration
list_items[0].find('p').find('span', {"class": "runtime"}).get_text()

'76 min'

In [25]:
# Extracting the genre
list_items[0].find('p').find('span', {"class": "genre"}).get_text().replace('\n', '')

'Animation, Family, Fantasy            '

In [29]:
# Extracting the rating
list_items[0].find('div', {"class": "ratings-bar"}).find('strong').get_text()

'8.0'

In [31]:
# list comprehension to extract the rest of the movies in the page
movie_title = [result.find('h3').find('a').get_text() for result in list_items]
year = [result.find('h3').find('span', {"class": "lister-item-year"}).get_text().replace('(', '').replace(')', '') for result in list_items]
duration = [result.find('p').find('span', {"class": "runtime"}).get_text() for result in list_items]
genre = [result.find('p').find('span', {"class": "genre"}).get_text().replace('\n', '') for result in list_items]
rating = [result.find('div', {"class": "ratings-bar"}).find('strong').get_text() for result in list_items]

In [34]:
# creating the dataframe
df = pd.DataFrame({'Movie Title': movie_title, 'Year':year, 'Duration':duration, 'Genre':genre, 'Rating':rating})
df

,Movie Title,Year,Duration,Genre,Rating
0,The Nightmare Before Christmas,1993,76 min,"Animation, Family, Fantasy",8.0
1,Los Locos Addams,1991,99 min,"Comedy, Fantasy",6.9
2,Blade Runner 2049,2017,164 min,"Action, Drama, Mystery",8.0
3,Harry Potter and the Sorcerer's Stone,2001,152 min,"Adventure, Family, Fantasy",7.6
4,Interstellar,2014,169 min,"Adventure, Drama, Sci-Fi",8.6
...,...,...,...,...,...
245,Richard Jewell,2019,131 min,"Biography, Crime, Drama",7.5
246,El padrino III,1990,162 min,"Crime, Drama",7.6
247,Papá por siempre,1993,125 min,"Comedy, Drama, Family",7.0
248,The Lovely Bones,2009,135 min,"Drama, Fantasy, Thriller",6.7


In [35]:
# save in csv file
df.to_csv('imdb.csv')